In [1]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

In [4]:
# Declaring Constants
IMAGE_PATH = "Sample_img.png"
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"

In [8]:
def preprocess_image(image_path):
  """ Loads image from path and preprocesses to make it model ready
      Args:
        image_path: Path to the image file
  """
  hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  # If PNG, remove the alpha channel. The model only supports
  # images with 3 color channels.
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

def save_image(image, filename):
  """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save.
  """
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s.jpg" % filename)
  print("Saved as %s.jpg" % filename)

processed_image = preprocess_image(IMAGE_PATH)
save_image(processed_image, "Sample_img.png")

In [10]:
%matplotlib inline
def plot_image(image, title=""):
  """
    Plots images from image tensors.
    Args:
      image: 3D image tensor. [height, width, channels].
      title: Title to display in the plot.
  """
  image = np.asarray(image)
  image = tf.clip_by_value(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  plt.imshow(image)
  plt.axis("off")
  plt.title(title)

In [11]:
model = hub.load(SAVED_MODEL_PATH)

Downloaded https://tfhub.dev/captain-pool/esrgan-tf2/1, Total size: 20.60MB



In [12]:
pip install tensorflow_hub

In [ ]:
import cv2
import os

def extract_frames(video_path, output_folder):
    # Create output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open video file
    cap = cv2.VideoCapture(video_path)

    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Loop through all frames
    for i in range(total_frames):
        # Read frame
        ret, frame = cap.read()
        if not ret:
            break

        # Save frame as image
        cv2.imwrite(os.path.join(output_folder, f"frame{i:06d}.png"), frame)

    # Release video file
    cap.release()


In [ ]:
extract_frames("/Video_Low_Resolu.mp4", "/out")

In [ ]:
import os
import tensorflow as tf
from PIL import Image

# Define input and output directories
input_dir = "/out"
output_dir = "/output"

# Loop through all the images in the input directory
for filename in os.listdir(input_dir):
    # Load the input image
    input_path = os.path.join(input_dir, filename)
    hr_image = preprocess_image(input_path)

    # Perform ISR using the ESRGAN model
    output_image = model(hr_image)

    # Remove extra dimension from output image array
    output_image = output_image[0]

    # Convert the image array to a PIL image
    output_image = tf.keras.preprocessing.image.array_to_img(output_image)
    output_image = output_image.resize((406,720), resample=Image.Resampling.BICUBIC)

    # Save the image to the output directory
    output_path = os.path.join(output_dir, filename)
    output_image.save(output_path)


In [ ]:
fps = 30.0
frame_size = (406, 720)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

In [ ]:
# Create a VideoWriter object
outing = cv2.VideoWriter('bcd.mp4', fourcc, fps, frame_size)


In [ ]:
# Get a list of all the frames in the folder
frame_folder = 'C:/Users/sidda/OneDrive/Desktop/output'
frames = [os.path.join(frame_folder, f) for f in os.listdir(frame_folder)]

# Loop through each frame and add it to the video
for frame in frames:
    # Read the frame image
    img = cv2.imread(frame)

    # Resize the frame to the video frame size
    img = cv2.resize(img, frame_size)

    # Write the frame to the video
    outing.write(img)

In [ ]:
outing

< cv2.VideoWriter 0000021312CF6FB0>

In [ ]:
# Release the VideoWriter and close any open windows
outing.release()
cv2.destroyAllWindows()
